In [1]:
import os 
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from PIL import Image

In [12]:
d = 'D:\\Projects\\CS_391R\\trajectories'

paths= []
for i in np.arange(0,21):
    traj='traj'+str(i)
    paths.append(os.path.join(d,traj))
    
for directory in paths:
    image_counter=0
    pose_df_path = os.path.join(directory,'pose.csv')
    pose_df = pd.read_csv(pose_df_path)
    image_df = pd.DataFrame(columns=['image_number','time'])
    for filename in os.listdir(directory):
        if filename.endswith('jpg'):
            image_file_path=os.path.join(directory,filename)
        else:
            continue
        image_creation_time=os.path.getctime(image_file_path)
        image_creation_time = (datetime.fromtimestamp(image_creation_time) - timedelta(hours=6))
        image_df= image_df.append({'image_number':image_counter,'time':image_creation_time},ignore_index=True)
        image_counter+=1
    image_df_path = os.path.join(directory,'image_creation_times.csv')
    image_df.to_csv(image_df_path,index=False)

In [13]:
d = 'D:\\Projects\\CS_391R\\trajectories'
paths= []
for i in np.arange(0,21):
    traj='traj'+str(i)
    paths.append(os.path.join(d,traj))
    
for directory in paths:
    image_counter=0
    pose_df_path = os.path.join(directory,'pose.csv')
    image_df_path = os.path.join(directory,'image_creation_times.csv')
    pose_df = pd.read_csv(pose_df_path)
    image_df = pd.read_csv(image_df_path)
    pose_df.columns= ['x','y','angle','time']
    pose_df=pose_df.dropna()
    pose_df['time']= pd.to_datetime(pose_df['time'], format='%Y-%m-%d.%H:%M:%S')
    image_df['time']= pd.to_datetime(image_df['time'], format='%Y-%m-%d %H:%M:%S.%f')
    image_df['time']= image_df['time'].astype('datetime64[s]')
    pose_df['time']= pose_df['time'].astype('datetime64[s]')
    print(directory)

    image_df.drop_duplicates('time', inplace = True)

    
    merged_df = pd.merge(image_df,pose_df,left_on='time', right_on='time')
    merged_df_path = os.path.join(directory,'merged.csv')
    merged_df.to_csv(merged_df_path,index=False)

D:\Projects\CS_391R\trajectories\traj0
D:\Projects\CS_391R\trajectories\traj1
D:\Projects\CS_391R\trajectories\traj2
D:\Projects\CS_391R\trajectories\traj3
D:\Projects\CS_391R\trajectories\traj4
D:\Projects\CS_391R\trajectories\traj5
D:\Projects\CS_391R\trajectories\traj6
D:\Projects\CS_391R\trajectories\traj7
D:\Projects\CS_391R\trajectories\traj8
D:\Projects\CS_391R\trajectories\traj9
D:\Projects\CS_391R\trajectories\traj10
D:\Projects\CS_391R\trajectories\traj11
D:\Projects\CS_391R\trajectories\traj12
D:\Projects\CS_391R\trajectories\traj13
D:\Projects\CS_391R\trajectories\traj14
D:\Projects\CS_391R\trajectories\traj15
D:\Projects\CS_391R\trajectories\traj16
D:\Projects\CS_391R\trajectories\traj17
D:\Projects\CS_391R\trajectories\traj18
D:\Projects\CS_391R\trajectories\traj19
D:\Projects\CS_391R\trajectories\traj20


In [10]:
d = 'D:\\Projects\\CS_391R\\trajectories'
paths= []
for i in np.arange(18,21):
    traj='traj'+str(i)
    paths.append(os.path.join(d,traj))
steps= 50
poses=[]
images=[]
velocity=[]
for directory in paths:
    
    merged_df_path = os.path.join(directory,'merged.csv')
    merged_df = pd.read_csv(merged_df_path)
    traj_count=0
    if merged_df.shape[0]>=steps:
        cutoff = merged_df.shape[0]//steps
        cutoff = cutoff*steps
        for index, row in merged_df.iterrows():
            if traj_count>=cutoff:
                break
            image_number= row['image_number']
            x = row['x']
            y = row['y']
            angle = row['angle']*(180/np.pi)
            image_number = str(image_number).zfill(4)
            image_path= 'left'+image_number+'.jpg'
            
            image_path = os.path.join(directory,image_path)
            image = Image.open(image_path)
     
            image = image.resize((32,32))
            np_image = np.array(image)
            np_image = np.dstack( ( np_image, np.zeros((32, 32)) ))
            images.append(np_image)
            poses.append(np.array([x,y,angle]))
            image.close()
            
            velocity.append(np.zeros((3)))
            traj_count+=1
images=np.array(images)
poses=np.array(poses)
velocity = np.array(velocity)
print(images.shape)
np.savez('gdc01_test.npz', vel=velocity,rgbd=images,pose=poses)

(450, 32, 32, 4)


In [11]:
data= np.load('gdc01_test.npz')
print(data.files)
vel = data['vel']
rgb=data['rgbd']
pose=data['pose']

print(rgb.shape,pose.shape)

print(rgb)
print('=================')
print(pose)
print('==================')

['vel', 'rgbd', 'pose']
(450, 32, 32, 4) (450, 3)
[[[[ 51.  56.  32.   0.]
   [ 56.  61.  34.   0.]
   [ 36.  38.  21.   0.]
   ...
   [101. 106.  52.   0.]
   [103. 108.  52.   0.]
   [ 75.  77.  35.   0.]]

  [[ 77.  88.  45.   0.]
   [ 88. 100.  48.   0.]
   [ 82.  94.  45.   0.]
   ...
   [102. 109.  52.   0.]
   [105. 111.  52.   0.]
   [ 76.  79.  37.   0.]]

  [[ 95. 106.  61.   0.]
   [106. 119.  63.   0.]
   [ 97. 110.  54.   0.]
   ...
   [105. 114.  56.   0.]
   [109. 116.  54.   0.]
   [ 78.  82.  39.   0.]]

  ...

  [[ 57.  60.  31.   0.]
   [ 93.  96.  48.   0.]
   [108. 109.  52.   0.]
   ...
   [ 75.  75.  35.   0.]
   [ 66.  67.  32.   0.]
   [ 52.  53.  26.   0.]]

  [[ 51.  48.  25.   0.]
   [108. 103.  51.   0.]
   [ 97. 100.  48.   0.]
   ...
   [ 62.  64.  33.   0.]
   [ 69.  69.  33.   0.]
   [ 62.  63.  29.   0.]]

  [[ 61.  50.  28.   0.]
   [ 91.  87.  52.   0.]
   [ 67.  68.  37.   0.]
   ...
   [ 33.  32.  19.   0.]
   [ 36.  35.  19.   0.]
   [ 38.  39.  1

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


labels = ['Collision Pedestrians', 'Collision Vehicles', 'Collision Layout', 'Red Light Violation']
men_means = [20, 34, 30, 35, 27]
women_means = [25, 32, 34, 20, 25]
TRANSFUSER=[]
AIM=[]
LBC=[]
OURS=[]

x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, men_means, width, label='Men')
rects2 = ax.bar(x + width/2, women_means, width, label='Women')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Scores')
ax.set_title('Scores by group and gender')
ax.set_xticks(x, labels)
ax.legend()

ax.bar_label(rects1, padding=3)
ax.bar_label(rects2, padding=3)

fig.tight_layout()

plt.show()